# Import related pacakages

In [1]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [2]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './data/iML1515.xml' 

# Reaction-kcat file.#s-1
# eg. AADDGT,49389.2889
reaction_kcat_file = "./data/reaction_kappori.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
model

Name,iML1515
Memory address,0x01bb0f73db38
Number of metabolites,1877
Number of reactions,3375
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [4]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [5]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [6]:
save_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [7]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [8]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
json_output_file="./model/iML1515_irr_enz_constraint_kapp.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [9]:
json_model_path="./model/iML1515_irr_enz_constraint_kapp.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
pfba_solution_df.to_csv('./analysis/ECMpy_kappori_solution_df_pfba.csv')
print(pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

norm_model = cobra.io.json.load_json_model(json_model_path)
norm_model_pfba_solution = cobra.flux_analysis.pfba(norm_model)
norm_model_pfba_solution_df = norm_model_pfba_solution.to_frame()
norm_model_pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
print(norm_model_pfba_solution_df.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.4191429334139575
0.8769972144269698


# Step4: Calibration parameters

Select calibration reaction

In [10]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
enzyme_amount = upperbound
percentage = 0.1 #10%
reaction_biomass_outfile = './analysis/reaction_biomass.csv'
select_value = 0.001
df_reaction_select = select_calibration_reaction(reaction_kcat_mw_file, json_model_path, enzyme_amount, percentage, reaction_biomass_outfile, select_value)
df_reaction_select

,biomass_diff,biomass_diff_ratio
PDH,0.015883,0.018110
AKGDH,0.009562,0.010903
GAPD,0.001564,0.001783


Calibration kcat

In [11]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
adj_kcat_title = 'smoment_adj_kcat'#'smoment_no_adj_kcat'#'kcat_GO'#'smoment_adj_kcat'#
change_kapp_file = "./analysis/reaction_kapp_change_round_1_cb.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change.csv"
need_change_reaction=list(df_reaction_select.index)

calibration_kcat(need_change_reaction, reaction_kcat_mw_file, json_model_path, adj_kcat_title, change_kapp_file, reaction_kapp_change_file)


In [12]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round1.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [13]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round1.json"
enz_model = get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
enz_model_pfba_solution = enz_model_pfba_solution.to_frame()
enz_model_pfba_solution.to_csv('./analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv')
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.47562259024946285


Compare with C13 data

In [14]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['MDH', 'AKGDH', 'SUCOAS_reverse', 'ICL', 'SUCDi', 'MALS']


Calibration kcat according c13 reaction list

In [15]:
reaction_kcat_mw_file = "./analysis/reaction_kapp_change.csv"
adj_kcat_title = 'smoment_adj_kcat'#'smoment_no_adj_kcat'#'kcat_GO'#'smoment_adj_kcat'
change_kapp_file = "./analysis/reaction_kapp_change_round_2_cb.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13.csv"

TCA_reactions=['CS','ACONTa','ACONTb','ICDHyr','MALS', 'MDH', 'ICL', 'SUCOAS_reverse', 'SUCDi', 'AKGDH']
calibration_kcat(TCA_reactions, reaction_kcat_mw_file, json_model_path, adj_kcat_title, change_kapp_file, reaction_kapp_change_file)


In [16]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round2.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [17]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round2.json"
enz_model = get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
enz_model_pfba_solution = enz_model_pfba_solution.to_frame()
enz_model_pfba_solution.to_csv('./analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv')
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.5098951540563751


In [18]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['PYK', 'PTAr', 'ACKr_reverse']


Calibration kcat according Enzyme usage 

In [19]:
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1 #kapp data sigma is 1
enz_total = round(ptot * f * sigma, 3)
reaction_flux_file = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_enz_usage_file = './analysis/ECMpy_kapp_adj_round2_reaction_enz_usage_df.csv'

reaction_enz_usage_df = get_enzyme_usage(enz_total,reaction_flux_file,reaction_kcat_mw_file,reaction_enz_usage_file)
reaction_enz_usage_df.head(10)

,kcat_mw,flux,enz useage,enz ratio
PDH,297.911591,7.082147,0.023773,0.104725
NADH16pp,1295.628496,23.273700,0.017963,0.079133
RBFSa,0.037137,0.000455,0.012247,0.053953
KARA2,12.484042,0.148139,0.011866,0.052274
KARA1_reverse,38.284397,0.445835,0.011645,0.051301
ENO,946.228175,9.008701,0.009521,0.041941
METS,13.180039,0.078707,0.005972,0.026307
FUM,867.934398,3.941657,0.004541,0.020006
FBA,478.270272,2.053945,0.004295,0.018919
GHMT2r,127.105741,0.540937,0.004256,0.018748


In [20]:
select_reaction = list(reaction_enz_usage_df[reaction_enz_usage_df['enz ratio']>0.01].index)#more than 2%
print(select_reaction)
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"

change_reaction_list=change_reaction_kcat_by_autopacmen(select_reaction,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list)

['PDH', 'NADH16pp', 'RBFSa', 'KARA2', 'KARA1_reverse', 'ENO', 'METS', 'FUM', 'FBA', 'GHMT2r', 'GLNS', 'METAT', 'ATPS4rpp', 'CS', 'PGI', 'ARGSS', 'GTPCI', 'GND', 'MOHMT', 'PGK_reverse', 'AKGDH']
['NADH16pp', 'RBFSa', 'KARA2', 'KARA1_reverse', 'ENO', 'METS', 'FUM', 'FBA', 'GHMT2r', 'GLNS', 'METAT', 'PGI', 'GTPCI', 'GND', 'MOHMT', 'PGK_reverse']


In [21]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round3.json"
f = 0.406
ptot = 0.56 
sigma = 1#kcapp数据，饱和度为1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [22]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round3.json"
enz_model = get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
enz_model_pfba_solution = enz_model_pfba_solution.to_frame()
enz_model_pfba_solution.to_csv('./analysis/ECMpy_kapp_adj_round3_solution_df_pfba.csv')
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.7894976064941117


In [23]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)#酶模型的ID ['PTAr_reverse', 'ACKr']

['PTAr', 'ACKr_reverse']


# Step4: Solveing enzyme concentration constraint by COBRApy.

In [24]:
#run enzyme constraint metabolic model
json_model_path="./model/iML1515_irr_enz_constraint_adj_round3.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.7894976064941117

In [25]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']


0.8769972144269698